In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, Robot, bgr8_to_jpeg
import torchvision
import torch
import torchvision.transforms as transforms
import cv2
import PIL.Image

In [2]:
camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=224, height=224)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [25]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 1)
model.load_state_dict(torch.load('best_steering_model_forward_merged.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval()

In [26]:
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    #image = transforms.functional.to_grayscale(image, num_output_channels=3)
    #image = transforms.functional.resized_crop(image, crop_percent * 224, 0, 224 - crop_percent * 224, 224, (224, 224))
    image = transforms.functional.to_tensor(image)
    image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return image[None, ...].to(device)

In [27]:
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, value=0.0, description='steering')

display(steering_slider)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [28]:
speed_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.22, description='speed')
steering_bias = widgets.FloatSlider(min=-0.3, max=0.3, step=0.0001, value=0.0, description='bias')

display(speed_slider)
display(steering_bias)

FloatSlider(value=0.22, description='speed', max=1.0, step=0.001)

FloatSlider(value=0.0, description='bias', max=0.3, min=-0.3, step=0.0001)

In [8]:
from jetbot import Robot

robot = Robot()

In [10]:
kp = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.22, description='kp')
kd = widgets.FloatSlider(min=0.0, max=1.0, step=0.001, value=0.37, description='kd')

display(kp, kd)

FloatSlider(value=0.22, description='kp', max=1.0, step=0.001)

FloatSlider(value=0.37, description='kd', max=1.0, step=0.001)

In [29]:
error = 0.0
error_last = 0.0

def execute(change):
    global error, error_last, error_i
    image = change['new']
    output = model(preprocess(image)).detach().cpu().numpy()
    steering = float(output[0][0]) + steering_bias.value
    steering_slider.value = steering 
    
    # compute error, derivative and integral
    error = - steering
    error_d = error - error_last
    error_last = error
    
    pid_error = kp.value * error + kd.value * error_d
    
    speed = speed_slider.value
    
    robot.set_motors(
        speed - pid_error,
        speed + pid_error
    )
    
execute({'new': camera.value})

In [30]:
camera.observe(execute, names='value')

In [24]:
import time

camera.unobserve_all()
time.sleep(0.5)
robot.stop()

In [ ]:
display(image_widget)
display(steering_slider)